<a href="https://colab.research.google.com/github/Shruti7001/-HD5210-Homework-/blob/main/midterm/midterm_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [20]:
import requests
import json
def priest(sex, age, resp_rate, oxygen_saturation, heart_rate, systolic_bp, temp, alertness, inspired_oxygen, performance_status):
    """
    Calculate the risk percentage based on various clinical parameters.

    Parameters:
        sex (str): The biological sex of the patient ('male' or 'female').
        age (int): The age of the patient in years.
        resp_rate (int): Respiratory rate in breaths per minute.
        oxygen_saturation (float): Oxygen saturation level (fraction, e.g., 0.96).
        heart_rate (int): Heart rate in beats per minute.
        systolic_bp (int): Systolic blood pressure in mmHg.
        temp (float): Body temperature in degrees Celsius.
        alertness (str): The level of alertness (e.g., 'alert', 'confused or not alert').
        inspired_oxygen (str): Oxygen supply mode (e.g., 'air', 'supplemental oxygen').
        performance_status (str): Patient's performance status.

    Returns:
        float: Risk percentage or None if an error occurs.

    Test Cases:
    >>> priest('male', 45, 18, 0.96, 85, 120, 37.0, 'alert', 'air', 'unrestricted normal activity')
    0.01
    >>> priest('female', 60, 25, 0.93, 100, 105, 38.5, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
    0.59
    >>> priest('male', 82, 30, 0.89, 140, 85, 39.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.99
    """
    score = 0

    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    if sex == 'female':
        score += 0
    elif sex == 'male':
        score += 1
    else:
        return None

    if 16 <= age <= 49:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4
    else:
        return None

    if resp_rate < 9:
        score += 3
    elif 9 <= resp_rate <= 11:
        score += 1
    elif 12 <= resp_rate <= 20:
        score += 0
    elif 21 <= resp_rate <= 24:
        score += 2
    elif resp_rate > 24:
        score += 3
    else:
        return None

    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3
    else:
        return None

    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3
    else:
        return None

    if systolic_bp < 91:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif 111 <= systolic_bp <= 219:
        score += 0
    elif systolic_bp > 219:
        score += 3
    else:
        return None

    if temp < 35.1:
        score += 3
    elif 35.1 <= temp <= 36.0:
        score += 1
    elif 36.1 <= temp <= 38.0:
        score += 0
    elif 38.1 <= temp <= 39.0:
        score += 1
    elif temp > 39.0:
        score += 2
    else:
        return None

    if alertness == 'alert':
        score += 0
    elif alertness == 'confused or not alert':
        score += 3
    else:
        return None

    if inspired_oxygen == 'air':
        score += 0
    elif inspired_oxygen == 'supplemental oxygen':
        score += 2
    else:
        return None

    if performance_status == 'unrestricted normal activity':
        score += 0
    elif performance_status == 'limited strenuous activity, can do light activity':
        score += 1
    elif performance_status == 'limited activity, can self-care':
        score += 2
    elif performance_status == 'limited self-care':
        score += 3
    elif performance_status == 'bed/chair bound, no self-care':
        score += 4
    else:
        return None

    if score <= 1:
        return 0.01
    elif score == 2 or score == 3:
        return 0.02
    elif score == 4:
        return 0.03
    elif score == 5:
        return 0.09
    elif score == 6:
        return 0.15
    elif score == 7:
        return 0.18
    elif score == 8:
        return 0.22
    elif score == 9:
        return 0.26
    elif score == 10:
        return 0.29
    elif score == 11:
        return 0.34
    elif score == 12:
        return 0.38
    elif score == 13:
        return 0.46
    elif score == 14:
        return 0.47
    elif score == 15:
        return 0.49
    elif score == 16:
        return 0.55
    elif 17 <= score <= 25:
        return 0.59
    elif score >= 26:
        return 0.99
    else:
        return None


In [21]:

import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    priest('male', 45, 18, 0.96, 85, 120, 37.0, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.01
ok
Trying:
    priest('female', 60, 25, 0.93, 100, 105, 38.5, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
Expecting:
    0.59
ok
Trying:
    priest('male', 82, 30, 0.89, 140, 85, 39.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.99
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [22]:
import requests

def find_hospital(age, sex, risk):
    """
    Determines the appropriate hospital for a patient based on age, sex, and risk level.

    Parameters:
        age (int): The age of the patient.
        sex (str): The biological sex of the patient ('male' or 'female').
        risk (float): The calculated risk score (between 0 and 1).

    Returns:
        str: The name of the hospital recommended for the patient.

    Test Cases:
    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(85, 'female', 0.75)
    'Wesley Woods Geriatric Hospital'
    >>> find_hospital(30, 'female', 0.3)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(90, 'male', 0.9)
    'Wesley Woods Geriatric Hospital'
    """
    if risk > 0.5:
        return 'Wesley Woods Geriatric Hospital'
    else:
        return 'Southwest Hospital and Medical Center'


In [23]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(85, 'female', 0.75)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(30, 'female', 0.3)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(90, 'male', 0.9)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [24]:
import requests
import json
def get_address(hospital_name):
    """
    Retrieve the address of the specified hospital by name.

    Test Cases:
    >>> get_address('Southwest Hospital and Medical Center')
    '501 FAIRBURN ROAD SW'

    >>> get_address('Wesley Woods Geriatric Hospital')
    '1821 CLIFTON ROAD NE'

    >>> get_address('Nonexistent Hospital')
    None
    """
    hospitals_data = {
        "Southwest Hospital and Medical Center": "501 FAIRBURN ROAD SW",
        "Wesley Woods Geriatric Hospital": "1821 CLIFTON ROAD, NE"
    }
    return hospitals_data.get(hospital_name, None)


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [25]:
import requests
import json

def process_people(file_url):
    """
    Processes patient data from a given file URL, calculates risk percentages, and determines appropriate hospital assignments.

    Parameters:
        file_url (str): The URL to a file containing patient data in a pipe-separated format.

    Returns:
        dict: A dictionary containing processed patient data with their respective risk percentage, hospital, and address.

    Summary:
        Prints the number of matched patient data, unmatched patient data, and skipped patients due to invalid data.

    Notes:
        The input file should have the following columns:
        PatientNumber|Sex|Age|RespRate|OxygenSaturation|HeartRate|SystolicBP|Temp|Alertness|InspiredOxygen|PerformanceStatus

    Test Cases:
        - Ensure the URL is reachable and data format is correct.
        - Handle cases where `priest` or `find_hospital` fails to compute values.

    Example:
        >>> results = process_people("http://example.com/patient_data.txt")
    """
    response = requests.get(file_url)
    if response.status_code != 200:
        return None

    people_data = response.text.splitlines()
    results = {}
    matched_count = 0
    unmatched_count = 0
    skipped_patients = 0

    for line in people_data[1:]:
        patient_data = line.split('|')

        patient_number = patient_data[0]
        sex = patient_data[1]
        age = int(patient_data[2])
        resp_rate = float(patient_data[3])
        oxygen_saturation = float(patient_data[4])
        heart_rate = float(patient_data[5])
        systolic_bp = float(patient_data[6])
        temp = float(patient_data[7])
        alertness = patient_data[8]
        inspired_oxygen = patient_data[9]
        performance_status = patient_data[10]

        # Calculate risk percentage using the priest function
        risk_percentage = priest(sex, age, resp_rate, oxygen_saturation, heart_rate, systolic_bp, temp, alertness, inspired_oxygen, performance_status)

        if risk_percentage is None:
            skipped_patients += 1
            print(f"Skipping patient {patient_number}: Unable to calculate risk percentage.")
            continue

        # Find hospital and address
        hospital = find_hospital(age, sex, risk_percentage)
        address = get_address(hospital) if hospital else None

        # Store the patient information
        patient_info = {
            "sex": sex,
            "age": age,
            "resp_rate": resp_rate,
            "oxygen_saturation": oxygen_saturation,
            "heart_rate": heart_rate,
            "systolic_bp": systolic_bp,
            "temp": temp,
            "alertness": alertness,
            "inspired_oxygen": inspired_oxygen,
            "performance_status": performance_status,
            "risk_percentage": risk_percentage,
            "hospital": hospital,
            "address": address
        }

        results[patient_number] = patient_info

        if hospital:
            matched_count += 1
        else:
            unmatched_count += 1

    # Print summary
    print(f"Number of matched patient data: {matched_count}")
    print(f"Number of unmatched patient data: {unmatched_count}")
    print(f"Number of skipped patients due to invalid data: {skipped_patients}")

    return results


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [26]:
import requests
import json

def process_people(file_url):
    response = requests.get(file_url)
    if response.status_code != 200:
        return None

    people_data = response.text.splitlines()
    results = {}
    matched_count = 0
    unmatched_count = 0
    skipped_patients = 0

    for line in people_data[1:]:
        patient_data = line.split('|')

        patient_number = patient_data[0]
        sex = patient_data[1]
        age = int(patient_data[2])
        resp_rate = float(patient_data[3])
        oxygen_saturation = float(patient_data[4])
        heart_rate = float(patient_data[5])
        systolic_bp = float(patient_data[6])
        temp = float(patient_data[7])
        alertness = patient_data[8]
        inspired_oxygen = patient_data[9]
        performance_status = patient_data[10]

        # Calculate risk percentage using the priest function
        risk_percentage = priest(sex, age, resp_rate, oxygen_saturation, heart_rate, systolic_bp, temp, alertness, inspired_oxygen, performance_status)

        if risk_percentage is None:
            skipped_patients += 1
            print(f"Skipping patient {patient_number}: Unable to calculate risk percentage.")
            continue

        # Find hospital and address
        hospital = find_hospital(age, sex, risk_percentage)
        address = get_address(hospital) if hospital else None

        # Store the patient information
        patient_info = {
            "sex": sex,
            "age": age,
            "resp_rate": resp_rate,
            "oxygen_saturation": oxygen_saturation,
            "heart_rate": heart_rate,
            "systolic_bp": systolic_bp,
            "temp": temp,
            "alertness": alertness,
            "inspired_oxygen": inspired_oxygen,
            "performance_status": performance_status,
            "risk_percentage": risk_percentage,
            "hospital": hospital,
            "address": address
        }

        results[patient_number] = patient_info

        if hospital:
            matched_count += 1
        else:
            unmatched_count += 1

    # Print summary
    print(f"Number of matched patient data: {matched_count}")
    print(f"Number of unmatched patient data: {unmatched_count}")
    print(f"Number of skipped patients due to invalid data: {skipped_patients}")

    return results

# Cell 6: Run the process_people function using the URL for the file containing people data
file_url = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'
process_people(file_url)

Number of matched patient data: 99
Number of unmatched patient data: 0
Number of skipped patients due to invalid data: 0


{'E9559': {'sex': 'FEMALE',
  'age': 40,
  'resp_rate': 24.0,
  'oxygen_saturation': 0.96,
  'heart_rate': 105.0,
  'systolic_bp': 115.0,
  'temp': 34.9,
  'alertness': 'ALERT',
  'inspired_oxygen': 'AIR',
  'performance_status': 'unrestricted normal activity',
  'risk_percentage': 0.15,
  'hospital': 'Southwest Hospital and Medical Center',
  'address': '501 FAIRBURN ROAD SW'},
 'E9385': {'sex': 'Female',
  'age': 51,
  'resp_rate': 19.0,
  'oxygen_saturation': 0.99,
  'heart_rate': 90.0,
  'systolic_bp': 91.0,
  'temp': 38.1,
  'alertness': 'ALERT',
  'inspired_oxygen': 'supplemental oxygen',
  'performance_status': 'unrestricted normal activity',
  'risk_percentage': 0.18,
  'hospital': 'Southwest Hospital and Medical Center',
  'address': '501 FAIRBURN ROAD SW'},
 'E3067': {'sex': 'female',
  'age': 40,
  'resp_rate': 29.0,
  'oxygen_saturation': 0.96,
  'heart_rate': 105.0,
  'systolic_bp': 95.0,
  'temp': 38.1,
  'alertness': 'ALERT',
  'inspired_oxygen': 'air',
  'performance_st

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---